In [ ]:
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/keras-team/keras/issues/2280#issuecomment-306959926
my_seed = 2024
import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "" # DISABLE CUDA, IT'S FASTER WITH CPU
# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(my_seed)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(my_seed)

# Force TensorFlow to use single thread. (to force it set the threads to 1)
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

session_conf = tf.ConfigProto(intra_op_parallelism_threads=0, inter_op_parallelism_threads=0)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(my_seed)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

rn.seed(my_seed)
#--- all other imports
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.constraints import max_norm
import keras.backend
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD, Adam
from numpy.random import seed
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.externals import joblib
from sklearn.model_selection import cross_validate
from sklearn.metrics import average_precision_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import copy
from bioinformatics_helpers.utils import get_mendelian_dataset
from bioinformatics_helpers.utils import hingesig_tf
from bioinformatics_helpers.balanced_generator import BalancedGenerator
from bioinformatics_helpers.utils import CustomKerasClassifier

In [ ]:
architecture = (100,)
dropout_rate = 0.2

In [ ]:
train_X, train_y, test_X, test_y = get_mendelian_dataset(return_df=True)

In [ ]:
def prc_score(y_true, y_pred):
    precision, recall, _ = precision_recall_curve(y_true=y_true, probas_pred=y_pred)
    return auc(x=recall, y=precision)

scoring = {
    'AVG_PREC': make_scorer(average_precision_score, needs_threshold=True),
    'AU_PRC' : make_scorer(prc_score, needs_threshold=True),
    'AU_ROC' : make_scorer(roc_auc_score, needs_threshold=True)
}

feature_per_example = train_X.shape[1]

In [ ]:
def create_model(architecture=(100,), dropout_rate=0.2, c=1e-15):
    model = Sequential()
    weights_initializer = keras.initializers.glorot_normal(seed=my_seed)
    bias_init = keras.initializers.RandomNormal(mean=0.1, stddev=0.05, seed=my_seed)
    input_dim = feature_per_example
    for units in architecture:
        model.add(
            Dense(
                units,
                input_dim = input_dim,
                kernel_initializer = weights_initializer,
                bias_initializer = bias_init,
                kernel_constraint=max_norm(c),
                activation="relu"
            )
        )
        model.add(
            Dropout(rate=dropout_rate, seed=my_seed)
        )
        input_dim=None # for the next layer keras infers its dimensions
        
    model.add(
        Dense(
            1,
            kernel_initializer=weights_initializer,
            bias_initializer=keras.initializers.zeros(),
            kernel_constraint=max_norm(c),
            activation='sigmoid'
    ))
    optimizer = Adam()
    model.compile(loss=hingesig_tf, optimizer=optimizer)
    return model

In [ ]:
params = {
    "model__negative_perc": [1],
    "model__positive_sample_perc": [1.5],
    "model__np_ratio": [5],
    "model__epochs" : [100],
    "model__architecture" : [architecture],
    "model__dropout_rate":[dropout_rate],
    "model__c": [3,3.5,4]


}

In [ ]:
gen = BalancedGenerator
model = CustomKerasClassifier(build_fn = create_model, generator=gen, verbose=0, shuffle=True)

pipe = Pipeline([("scaler", StandardScaler()), ("model",model)])
cv = StratifiedKFold(n_splits=5, random_state=my_seed, shuffle=True)
grid_search = GridSearchCV(estimator=pipe,param_grid=params,
                           scoring=scoring,
                           return_train_score=True,
                           cv=cv,
                           refit=False,
                           verbose=20
                           )
grid_search.fit(train_X, train_y)
#saving cv_results_
cv_results = pd.DataFrame.from_dict(grid_search.cv_results_)
cv_results.to_csv("cv_results_scaler_adam_less_features_with_dropout_mb_gen_maxnorm.csv")
